In [1]:
from langsmith import Client
from langsmith.schemas import Run, Example
from langsmith.evaluation import evaluate
import openai
from langsmith.wrappers import wrap_openai

from dotenv import load_dotenv
import os

load_dotenv()


client = Client()

# Define dataset: these are your test cases
dataset_name = "neurapolis-file-sections"
# dataset = client.create_dataset(dataset_name, description="Rap battle prompts.")
# client.create_examples(
#     inputs=[
#         {"question": "a rap battle between Atticus Finch and Cicero"},
#         {"question": "a rap battle between Barbie and Oppenheimer"},
#     ],
#     outputs=[
#         {"must_mention": ["lawyer", "justice"]},
#         {"must_mention": ["plastic", "nuclear"]},
#     ],
#     dataset_id=dataset.id,
# )

# Define AI system
openai_client = wrap_openai(openai.Client())

def boilterplate_evaluator(inputs: dict) -> dict:
    messages = [{"role": "user", "content": inputs["question"]}]
    response = openai_client.chat.completions.create(messages=messages, model="gpt-3.5-turbo")
    return {"output": response}


from langchain_openai import ChatOpenAI
# Define evaluators
def split_correctly(run: Run, example: Example) -> dict:
    
    prompt = """

You are tasked with evaluating a submission based on the provided original file text and the assigned sectionizer type. You will also receive the first 500 characters of each section generated by the submission. Your evaluation involves two tasks:

Task 1: Validate Sectionizer Type

• Objective: Determine if the assigned sectionizer type is correct for the given file.

• Possible Types:

• NOT_RELATED_TOPICS: The document contains multiple unrelated topics, often seen in meeting minutes with varied agenda items.

• RELATED_TOPICS: The document revolves around a single main topic, possibly with several subtopics related to that central theme.

• OTHER: The document requires further analysis or pertains to a topic that cannot be extracted from text alone (e.g., election results).

• Output: Provide a boolean (True/False) indicating whether the sectionizer type is correct, along with a brief justification for your decision.

Task 2: Evaluate Document Splitting

• Objective: Verify if the document was split correctly based on the assigned sectionizer type.

• Criteria:

• The document should only be split into multiple sections if the sectionizer type is NOT_RELATED_TOPICS.

• Evaluate the logical placement of the splits, if applicable.

• Output: Confirm whether the document was split correctly (True/False) and provide a brief explanation.

Data Provided:

[BEGIN DATA]

[Original File Text and sectionizer type]:

{input}

[Generated Sections (First 500 Characters Each)]:

{submission}

[END DATA]

"""


    print(example.inputs)
    print(example.outputs)
    
    file = str(example.inputs)
    sections = str(example.outputs)
    
    print("\033[94m" + file + "\033[0m")
    print("\033[94m" + sections + "\033[0m")

    llm = ChatOpenAI(model="gpt-3.5-turbo")
    
    prompt = prompt.format(input=file, submission=sections)
    response = llm.invoke(prompt)
    
    # Parse the response content to extract required information
    content = response.content
    
    # Extract boolean values and reasons from the content
    is_sectionizer_type_correct = "True" in content.split("Task 1:")[1].split("Task 2:")[0]
    is_splitting_correct = "True" in content.split("Task 2:")[1]
    
    sectionizer_type_correctness_reason = content.split("Task 1:")[1].split("Task 2:")[0].split("\n", 1)[1].strip()
    splitting_correctness_reason = content.split("Task 2:")[1].split("\n", 1)[1].strip()
    
    # Construct the result dictionary
    result = {
        "key": "split_correctly",
        "score": float(is_splitting_correct),
        "value": {
            "is_sectionizer_type_correct": is_sectionizer_type_correct,
            "sectionizer_type_correctness_reason": sectionizer_type_correctness_reason,
            "is_splitting_correct": is_splitting_correct,
            "splitting_correctness_reason": splitting_correctness_reason
        }
    }
    
    return result

experiment_results = evaluate(
    boilterplate_evaluator, # Your AI system
    data=dataset_name, # The data to predict and grade over
    evaluators=[split_correctly], # The evaluators to score the results
    experiment_prefix="rap-generator", # A prefix for your experiment names to easily identify them
    client=client,
    metadata={
      "version": "1.0.0",
    },
)

/Users/pascal/neurapolis/etl-evaluator/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


View the evaluation results for experiment: 'rap-generator-2dc8913c' at:
https://eu.smith.langchain.com/o/f2baf51a-5907-4625-b2ab-a98883dd8671/datasets/20233d7f-efe8-40b1-afbb-457beb5f271f/compare?selectedSessions=c9dc45b0-306c-402f-a6ae-3ae4d5776606




0it [00:00, ?it/s]Error running target function: 'question'
Error running target function: 'question'
Error running target function: 'question'
Error running target function: 'question'
Error running target function: 'question'
Error running target function: 'question'
Error running target function: 'question'
Error running target function: 'question'
Error running target function: 'question'
Error running target function: 'question'
Error running target function: 'question'
Error running target function: 'question'
Error running target function: 'question'
Error running target function: 'question'
Error running target function: 'question'
Error running target function: 'question'
Error running target function: 'question'
Error running target function: 'question'
Error running target function: 'question'
Error running target function: 'question'
Error running target function: 'question'
Error running target function: 'question'
Error running target function: 'question'
Error running ta

{'file': 'Das Bürgermeisteramt der Stadt Freiburg im Breisgau - Dezernat I -\n\nFreiburg i. Br., 16.02.2024  \nTel.: 0761/201-1110  \nHerr Knobloch\n\n2. Sitzung des Gemeinderates\n\nMitglieder des Gemeinderates\n\nIch lade zu der am\n\nDienstag, 27. Februar 2024, 18:00 Uhr\n\nim Neuen Ratssaal des Rathauses stattfindenden Sitzung des Gemeinderates ein.\n\nT a g e s o r d n u n g\n\nÖffentlicher Teil\n\n1. Bebauungsplan mit örtlichen Bauvorschriften und Ausgleichsflächen auf den Gemarkungen Freiburg, Lehen, Waltershofen, Opfingen und Hochdorf "Dietenbach - Am Frohnholz", Plan-Nr. 6-175  \n   a) Entscheidung über die im Rahmen der ersten förmlichen Öffentlichkeits- und Behördenbeteiligung eingegangenen Stellungnahmen  \n   b) Beschluss des Planentwurfs für die erneute förmliche Öffentlichkeits- und Behördenbeteiligung (Offenlagebeschluss) und das weitere Verfahren  \n   c) Beschluss des zentralen Versorgungsbereichs "Stadtteilzentrum Dietenbach"  \n   d) Beschluss zur Durchführung vorge

Error running evaluator <DynamicRunEvaluator split_correctly> on run fe5ca03b-40ba-42f9-b1ea-6abe93a37725: BadRequestError('Error code: 400 - {\'error\': {\'message\': "Sorry! We\'ve encountered an issue with repetitive patterns in your prompt. Please try again with a different prompt.", \'type\': \'invalid_request_error\', \'param\': \'prompt\', \'code\': \'invalid_prompt\'}}')
Traceback (most recent call last):
  File "/Users/pascal/neurapolis/etl-evaluator/.venv/lib/python3.12/site-packages/langsmith/evaluation/_runner.py", line 1323, in _run_evaluators
    evaluator_response = evaluator.evaluate_run(
                         ^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/pascal/neurapolis/etl-evaluator/.venv/lib/python3.12/site-packages/langsmith/evaluation/evaluator.py", line 327, in evaluate_run
    result = self.func(
             ^^^^^^^^^^
  File "/Users/pascal/neurapolis/etl-evaluator/.venv/lib/python3.12/site-packages/langsmith/run_helpers.py", line 629, in wrapper
    raise e
  Fil

...

{'file': "Öffentliche Sitzung\n\ndes Ortschaftsrates Munzingen\n\nam 08.12.2021\n\nin der Schloßbuckhalle in Munzingen in Freiburg im Breisgau\n\nErgebnismitteilung\n\nAnwesend:\n\nOrtschaftsrätinnen und Ortschaftsräte mit Stimmrecht:\nOrtsvorsteher Christian Schildecker\nOrtschaftsrätin Elisabetha Gräfin von Kageneck\nOrtschaftsrat Clemens Lang\nOrtschaftsrätin Liliane Moser\nOrtschaftsrat Klaus Riemann\nOrtschaftsrat Philipp Schillinger\nOrtschaftsrat Wolfgang Schmid\nOrtschaftsrat Ulrich Schopp\nOrtschaftsrätin Kunigunde Stifel\nOrtschaftsrätin Katja Weber\nOrtschaftsrätin Katharina Wirth\n\nEs fehlten:\nOrtschaftsrat Toni Luhr entschuldigt\n\nGäste / Sachkundige und Sachverständige:\nReferent Herr Martin Bornhauser, 1. Betriebsleiter des Eigenbetrieb Friedhöfe (EBF), App. 6600, sowie Herr Michael Hacker, Geschäftsführer des Abwasserzweckverbandes Breisgauer Bucht (AZV)\nBZ: Herr Schindler hat sich entschuldigt\n\nVon der Verwaltung: Beate Eicher, Schriftführerin\n\nZur 11. Sitzung 

24it [00:06,  4.72it/s]

{'file': 'Öffentliche Sitzung\n\ndes Ortschaftsrates Munzingen\n\nam 10.02.2021\n\nin der Schloßbuckhalle in Munzingen in Freiburg im Breisgau\n\nErgebnismitteilung\n\nAnwesend:\n\nOrtschaftsrätinnen und Ortschaftsräte mit Stimmrecht:\n\nOrtsvorsteher Christian Schildecker  \nOrtschaftsrat Clemens Lang  \nOrtschaftsrat Toni Luhr  \nOrtschaftsrat Klaus Riemann  \nOrtschaftsrat Philipp Schillinger  \nOrtschaftsrat Wolfgang Schmid  \nOrtschaftsrat Ulrich Schopp  \nOrtschaftsrätin Kunigunde Stifel  \nOrtschaftsrätin Katja Weber  \nOrtschaftsrätin Katharina Wirth  \nOrtsverwaltung Munzingen\n\nEs fehlte:\n\nOrtschaftsrätin Elisabetha Gräfin von Kageneck entschuldigt wg. Krankheit  \nOrtschaftsrätin Liliane Moser entschuldigt\n\nGäste / Sachkundige und Sachverständige: keine\n\nVon der Verwaltung: Beate Eicher, Schriftführerin\n\nZur 02. Sitzung des Ortschaftsrates Munzingen wurde form- und fristgerecht eingeladen.\n\nBei Eröffnung der regulären öffentlichen Sitzung um 19:30 Uhr durch OV Sch

42it [00:11,  3.77it/s]


Failed to batch ingest runs: langsmith.utils.LangSmithConnectionError: Connection error caused failure to POST https://eu.api.smith.langchain.com/runs/batch in LangSmith API. Please confirm your LANGCHAIN_ENDPOINT. ConnectionError(ProtocolError('Connection aborted.', TimeoutError('The write operation timed out')))
Content-Length: 20840230
API Key: lsv2_********************************************5c
post: trace=5be1f6f7-2324-41b1-b866-8ad963e7188e,id=5be1f6f7-2324-41b1-b866-8ad963e7188e; trace=57388f2a-d72d-4b33-b348-08a436da2e98,id=57388f2a-d72d-4b33-b348-08a436da2e98; trace=d9bf588f-b21f-4088-a03c-948e8f7157d5,id=d9bf588f-b21f-4088-a03c-948e8f7157d5; trace=2dbfb9ce-fcc0-4f38-add8-e7822ff7fc26,id=2dbfb9ce-fcc0-4f38-add8-e7822ff7fc26; trace=d8891be4-d7c3-425f-8089-876f51707e83,id=d8891be4-d7c3-425f-8089-876f51707e83; trace=6d8e8a19-f875-4efa-b3c9-dd1358e3f6bc,id=6d8e8a19-f875-4efa-b3c9-dd1358e3f6bc; trace=fe5ca03b-40ba-42f9-b1ea-6abe93a37725,id=fe5ca03b-40ba-42f9-b1ea-6abe93a37725; trac